# EJFAT LB Control Plane Tester

This notebook stands up a slice of 3 nodes - sender, receiver and cpnode. The control plane daemon is deployed on the `cpnode` node in a 'mock' configuration (with no FPGA). DAQ and worker node code can be deployed on `sender` and `receiver` nodes for testing. The slice uses 'shared' and is created within a single FABRIC site for simplicity. It uses a single L2 bridge connection with RFC1918 IPv4 addressing, allowing all nodes to talk to each other. It is possible to run the dataplane assuming a single worker can keep up with a single sender since no actual load balancer is present in this configuration.

## Preparation

- Be sure to download the latest [UDPLBd release](https://github.com/esnet/udplbd) (since the repo is private it is hard to download it from inside the slice) and put it under `code/` folder next to this notebook.
- Update the name of the downloaded file (`udplbd_file`) in the first cell as needed

## Preamble

In [ ]:
# Preamble
from datetime import datetime
from datetime import timezone
from datetime import timedelta

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

fablib = fablib_manager()             
fablib.show_config();

# variable settings
slice_name = 'UDP LB Control Plane Testing slice'
# for each node specify IP address (assuming /24), OS image
# note that most of the keys in these dictionaries map directly
# onto parameters to add_node()
node_config = {
    'sender': {
        'ip':'192.168.0.1', 
        'image':'default_ubuntu_22',
        'cores': 8,
        'ram': 8,
        'disk': 100 },
    'recver': {
        'ip':'192.168.0.2', 
        'image':'default_ubuntu_22',
        'cores':8,
        'ram': 8,
        'disk': 100 },
    'cpnode': {
        'ip':'192.168.0.3', 
        'image':'docker_ubuntu_22',
        'cores':8,
        'ram': 8,
        'disk': 100 },
}
# skip these keys as they are not part of add_node params
skip_keys = ['ip']
# this is the NIC to use
nic_model = 'NIC_Basic'
# the subnet should match IPs
subnet = IPv4Network("192.168.1.0/24")

# if you want to force a site instead of using random
site_override = 'KANS'
#site_override = None

# UDPLBd code location in the Jupyter container
udplbd_file = 'code/udplbd-main-05-17-24.zip'

## Create the slice

In [ ]:
# list all slices I have running
output_dataframe = fablib.list_slices(output='pandas')
if output_dataframe:
    print(output_dataframe)
else:
    print('No active slices under this project')

In [ ]:
# List available images
available_images = fablib.get_image_names()

print(f'Available images are: {available_images}')

In [ ]:
# find an available site in continental US
lon_west=-124.3993243
lon_east=-69.9721573

# getting a random site make take a bit of time
if not site_override:
    selected_site = fablib.get_random_site(filter_function=lambda x: x['location'][1] < lon_east
                                              and x['location'][1] > lon_west) 
else:
    selected_site = site_override

if selected_site:
    print(f'Selected site is {selected_site}')
else:
    print('Unable to find a site matching the requirements')

# write selected site into node attributes
for n in node_config:
    node_config[n]['site'] = selected_site
    

In [ ]:
# build a slice
slice = fablib.new_slice(name=slice_name)

# create a network
net1 = slice.add_l2network(name='site_bridge_net', subnet=subnet)

nodes = dict()
# create  nodes for sending and receiving with a selected network card
# use subnet address assignment
for node_name, node_attribs in node_config.items():
    print(f"{node_name=} {node_attribs['ip']}")
    nodes[node_name] = slice.add_node(name=node_name, **{x: node_attribs[x] for x in node_attribs if x not in skip_keys})
    nic_interface = nodes[node_name].add_component(model=nic_model, name='_'.join([node_name, nic_model, 'nic'])).get_interfaces()[0]
    net1.add_interface(nic_interface)
    nic_interface.set_mode('config')
    nic_interface.set_ip_addr(node_attribs['ip'])
    # postboot configuration is under 'post-boot' directory
    nodes[node_name].add_post_boot_upload_directory('post-boot','.')
    nodes[node_name].add_post_boot_execute(f'chmod +x post-boot/{node_name}.sh && ./post-boot/{node_name}.sh')

# Submit the slice
slice.submit();

## Start the container

In [ ]:
# get slice details (if not creating new)
slice = fablib.get_slice(name=slice_name)
a = slice.show()
nets = slice.list_networks()
nodes = slice.list_nodes()

cpnode = slice.get_node(name="cpnode")    

In [ ]:
# check if any dockers are running already and that we have compose and buildx installed by post-boot script
commands = [
    'docker container ls',
    'docker compose version',
    'docker buildx version'
]
for command in commands:
    print(f'\tExecuting "{command}"')
    #stdout, stderr = cpnode.execute(command, quiet=True, output_file=node.get_name() + '_install.log')
    stdout, stderr = cpnode.execute(command)
    if not stderr and len(stderr) > 0:
        print(f'Error encountered with "{command}": {stderr}')

In [ ]:
# upload the mock config file for UDPLBd
result = cpnode.upload_file('config/lb_mock.yml','lb_mock.yml')

# upload the UDPLBd zip file (from github)
# note that UDPLBd repo is private, so you need to download whatever release you need as .zip
# under 'code' directory in your Jupyter continer
result = cpnode.upload_file(udplbd_file, 'udplbd-main.zip')

In [ ]:
# start the UDPLBd container

commands = [
    f'unzip udplbd-main.zip',
    f'mv udplbd-main udplbd',
    f'cp lb_mock.yml ./udplbd/etc/config.yml',
    f'cd udplbd; docker compose up -d'
]

for command in commands:
    print(f'\tExecuting "{command}"')
    #stdout, stderr = cpnode.execute(command, quiet=True, output_file=node.get_name() + '_install.log')
    stdout, stderr = cpnode.execute(command)
    if not stderr and len(stderr) > 0:
        print(f'Error encountered with "{command}": {stderr}')

In [ ]:
# check the logs
commands = [
    'docker compose ls',
    'cd udplbd; docker compose logs'
]

for command in commands:
    print(f'\tExecuting "{command}"')
    #stdout, stderr = cpnode.execute(command, quiet=True, output_file=node.get_name() + '_install.log')
    stdout, stderr = cpnode.execute(command)
    if not stderr and len(stderr) > 0:
        print(f'Error encountered with "{command}": {stderr}')

## Manage the slice

### Extend

In [ ]:
# Set end host to now plus 14 days
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

### Delete

In [ ]:
slice = fablib.get_slice(slice_name)
slice.delete()